In [138]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup
from flask import Flask, render_template, redirect, url_for
from flask_pymongo import PyMongo
import pymongo
import datetime
from webdriver_manager.chrome import ChromeDriverManager

In [139]:
# to setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\lcswi\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


## NASA Mars News - Scrape the Mars News Site and collect the latest News Title and Paragraph Text

In [140]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)

In [141]:
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [142]:
news_title = soup.find_all('div', class_='content_title')[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print('---------------------------------------------------------------------------')
print(news_p)

NASA's Perseverance Rover Mission Getting in Shape for Launch
---------------------------------------------------------------------------
Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. 


## JPL Mars Space Images - Featured Image 

In [143]:
# URL of page to be scraped
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [144]:
# Create BeautifulSoup object; parse with 'html.parser'
html = browser.html
soup_JPL = BeautifulSoup(html, 'html.parser')

In [145]:
# identify and return the url string for the featured_image_url 
image_path = soup_JPL.find_all('img')[1]['src']
featured_image_url = url + image_path
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars2.jpg


## MARS Facts - Table Mars facts including diameter, mass, etc

In [194]:
# We can use the read_html function in Pandas to automatically scrape any tabular data from a page.
url = 'https://galaxyfacts-mars.com'
tables = pd.read_html(url)
table = tables[0]
table.columns = ['Mars-Earth Comparison', 'Mars', 'Earth']
Mars_Table = table.drop(index=0)
Mars_Table

,Mars-Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [203]:
# comparison table converted from dataframe to html table (note: all "\n" - unwanted news lines, have been dropped from html table)
html_table = Mars_Table.to_html()
clean_html_table = html_table.replace('\n', '')
clean_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars-Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>4</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>5</th>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>6</th>      <td>Temperature:</td>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [201]:
# Saving the Mars Earth Comparison table directly to an HTML table string called "table.html"
Mars_Table_HTML = Mars_Table.to_html('table.html')